# Sentence Retrieval

In [99]:
import json,math,nltk

train_file = open("train.json",'r')
dev_file = open("dev.json",'r')
test_file=open("dev.json",'r')
train = json.loads(train_file.read())
dev = json.loads(dev_file.read())
test = json.loads(test_file.read())
# train[0]['sentences']
# train[0]['qa']

In [100]:
from nltk.corpus import stopwords
# may be stem?

lemmatizer = nltk.stem.wordnet.WordNetLemmatizer()
word_tokenizer = nltk.tokenize.regexp.WordPunctTokenizer()
stopword =  stopwords.words()

def lemmatize(word):
    lemma = lemmatizer.lemmatize(word,'v')
    if lemma == word:
        lemma = lemmatizer.lemmatize(word,'n')
    return lemma

def doc_word_dict(doc):
    word_dict = set()
    for sent in doc['sentences']:
        for word in  word_tokenizer.tokenize(sent):
            word = lemmatize(word.lower())
            if word not in stopword:
                word_dict.add(word)
    return word_dict

In [101]:
def get_BOW(sent):
    term_dict={}
    for word in word_tokenizer.tokenize(sent):
        word = lemmatize(word.lower())
        if word not in stopword:
            term_dict[word]=term_dict.setdefault(word,0)+1
    return term_dict

def cal_BOW(doc):
    doc_term_matrix = [] 
    for sent in doc['sentences']:
        temp = get_BOW(sent)
        doc_term_matrix.append(temp)
    return doc_term_matrix

In [110]:
from sklearn.feature_extraction import DictVectorizer
from sklearn.feature_extraction.text import TfidfTransformer
from scipy.spatial.distance import cosine as cos_distance

vectorizer = DictVectorizer()
transformer = TfidfTransformer(smooth_idf=False,norm=None)

term_matrix = transformer.fit_transform(vectorizer.fit_transform(cal_BOW(train[0])))
query =  transformer.transform(vectorizer.transform(get_BOW(train[0]['qa'][0]['question'])))

In [104]:
def get_best_doc_num(query):
    query =  transformer.transform(vectorizer.transform(get_BOW(query)))
    f = lambda x: cos_distance(query.toarray(),term_matrix[x].toarray())
    best_doc = min(range(term_matrix.shape[0]),key=f)
    return best_doc

In [112]:
vectorizer = DictVectorizer()
transformer = TfidfTransformer(smooth_idf=False,norm=None)

correct_rate = []
# for doc in train[0]:
doc_correct_rate = []
term_matrix = transformer.fit_transform(vectorizer.fit_transform(cal_BOW(train[0])))
for qa in train[0]['qa']:
    doc_correct_rate.append(get_best_doc_num(qa['question']))
correct_rate.append(doc_correct_rate)

In [113]:
correct_rate

[[325,
  310,
  31,
  137,
  152,
  3,
  5,
  6,
  7,
  243,
  14,
  10,
  12,
  11,
  11,
  13,
  14,
  103,
  4,
  407,
  102,
  407,
  356,
  34,
  37,
  39,
  40,
  39,
  37,
  90,
  148,
  45,
  44,
  42,
  30,
  30,
  47,
  46,
  191,
  52,
  30,
  90,
  54,
  401,
  58,
  58,
  59,
  72,
  388,
  61,
  62,
  64,
  262,
  284,
  74,
  74,
  138,
  310,
  71,
  76,
  82,
  76,
  453,
  138,
  92,
  35,
  27,
  88,
  92,
  89,
  90,
  48,
  92,
  113,
  105,
  103,
  98,
  254,
  108,
  115,
  108,
  451,
  112,
  112,
  113,
  112,
  112,
  111,
  115,
  105,
  117,
  338,
  138,
  118,
  121,
  382,
  119,
  122,
  122,
  125,
  126,
  122,
  128,
  138,
  118,
  138,
  138,
  131,
  131,
  131,
  131,
  134,
  382,
  137,
  310,
  24,
  136,
  308,
  113,
  113,
  184,
  133,
  260,
  148,
  149,
  178,
  150,
  151,
  157,
  153,
  155,
  154,
  154,
  157,
  138,
  182,
  162,
  161,
  138,
  164,
  167,
  164,
  164,
  164,
  168,
  169,
  169,
  259,
  172,
  174,
  169,
  1

# Entity Extraction

In [18]:
st.tag(word_tokenizer.tokenize("Located in the northwestern Pacific Ocean, Guam is one of five American territories with an established civilian government in 1980s."))
# st.tag("Located in the northwestern Pacific Ocean, Guam is one of five American territories with an established civilian government in 1980s.".split())

[(u'Located', u'O'),
 (u'in', u'O'),
 (u'the', u'O'),
 (u'northwestern', u'O'),
 (u'Pacific', u'LOCATION'),
 (u'Ocean', u'LOCATION'),
 (u',', u'O'),
 (u'Guam', u'LOCATION'),
 (u'is', u'O'),
 (u'one', u'O'),
 (u'of', u'O'),
 (u'five', u'O'),
 (u'American', u'O'),
 (u'territories', u'O'),
 (u'with', u'O'),
 (u'an', u'O'),
 (u'established', u'O'),
 (u'civilian', u'O'),
 (u'government', u'O'),
 (u'in', u'O'),
 (u'1980s', u'DATE'),
 (u'.', u'O')]

In [88]:
# import time

# start = time.time()
# test_sent_list = []
# for i in range(len(correct_rate)):
#     for j in list(set(correct_rate[i])):
#         tmp = st.tag(test[i]['sentences'][j].split())
#         test_sent_list.append(tmp)
# print time.time()-start

452.405480146


In [220]:
from nltk.tag.stanford import StanfordNERTagger
st = StanfordNERTagger('/usr/share/stanford-ner/classifiers/english.muc.7class.distsim.crf.ser.gz',
               '/usr/share/stanford-ner/stanford-ner.jar') 

start = time.time()
train_sent_list = []
for i in range(len(correct_rate)):
    for j in list(set(correct_rate[i])):
        train_sent_list.append(train[i]['sentences'][j].split())
train_sent_list = st.tag_sents(train_sent_list) 
print time.time()-start

2.99265599251


In [221]:
def tune_other(train_sent_list):
    for i in range(len(train_sent_list)):
        for j in range(len(train_sent_list[i])):
            term,tag = train_sent_list[i][j]
            if tag == "ORGANIZATION"  or ((term,tag)!=train_sent_list[i][0] and tag == 'O' and term[0].isupper()):
                train_sent_list[i][j] = (term,"OTHER")

tune_other(train_sent_list)

In [224]:
train_sent_list

[[(u'The', u'O'),
  (u'phonograph', u'O'),
  (u'disc', u'O'),
  (u'record', u'O'),
  (u'was', u'O'),
  (u'the', u'O'),
  (u'primary', u'O'),
  (u'medium', u'O'),
  (u'used', u'O'),
  (u'for', u'O'),
  (u'music', u'O'),
  (u'reproduction', u'O'),
  (u'until', u'O'),
  (u'late', u'O'),
  (u'in', u'O'),
  (u'the', u'O'),
  (u'20th', u'O'),
  (u'century,', u'O'),
  (u'replacing', u'O'),
  (u'the', u'O'),
  (u'phonograph', u'O'),
  (u'cylinder', u'O'),
  (u'record\u2013with', u'O'),
  (u'which', u'O'),
  (u'it', u'O'),
  (u'had', u'O'),
  (u'co-existed', u'O'),
  (u'from', u'O'),
  (u'the', u'O'),
  (u'late', u'O'),
  (u'1880s', u'O'),
  (u'through', u'O'),
  (u'to', u'O'),
  (u'the', u'O'),
  (u'1920s\u2013by', u'O'),
  (u'the', u'O'),
  (u'late', u'O'),
  (u'1920s.', u'O')],
 [(u'Records', u'O'),
  (u'retained', u'O'),
  (u'the', u'O'),
  (u'largest', u'O'),
  (u'market', u'O'),
  (u'share', u'O'),
  (u'even', u'O'),
  (u'when', u'O'),
  (u'new', u'O'),
  (u'formats', u'O'),
  (u'such', u

In [223]:
def combine_entity(train_sent_list):
    for i in range(len(train_sent_list)):
        j = 0
        while j < len(train_sent_list[i])-2:
            term,tag = train_sent_list[i][j]
            term_n,tag_n = train_sent_list[i][j+1]
            if tag == tag_n and tag != "O":
                temp =  (term + " " + term_n,tag)
                train_sent_list[i][j] = temp
                del train_sent_list[i][j+1]
            j += 1

combine_entity(train_sent_list)

In [225]:
def hasNumbers(inputString):
    return any(char.isdigit() for char in inputString)

tags = ["PERSON","LOCATION","NUMBER","OTHER"]
entity_pool = []
for i in train_sent_list:
    sent_tag_dict = dict.fromkeys(tags,[])
    for j in range(len(i)):
        term,tag = i[j]
        if tag == "PERSON" or tag == "LOCATION" or tag == "OTHER":
            sent_tag_dict[tag] = sent_tag_dict[tag]+ [term]
        elif tag == "DATE" or tag == "TIME" or tag == "PERCENT" or hasNumbers(term):
            sent_tag_dict["NUMBER"] = sent_tag_dict["NUMBER"]+ [term]
    entity_pool.append(sent_tag_dict)

In [226]:
entity_pool

[{'LOCATION': [],
  'NUMBER': [u'20th', u'1880s', u'1920s\u2013by', u'1920s.'],
  'OTHER': [],
  'PERSON': []},
 {'LOCATION': [], 'NUMBER': [], 'OTHER': [], 'PERSON': []},
 {'LOCATION': [], 'NUMBER': [u'1980s,', u'1991.'], 'OTHER': [], 'PERSON': []},
 {'LOCATION': [], 'NUMBER': [u'1990s', u'2010s,'], 'OTHER': [], 'PERSON': []},
 {'LOCATION': [u'the U.S.'],
  'NUMBER': [u'21st', u'9.2', u'2014,', u'260%', u'2009.'],
  'OTHER': [],
  'PERSON': []},
 {'LOCATION': [], 'NUMBER': [u'2000s,'], 'OTHER': [], 'PERSON': []},
 {'LOCATION': [],
  'NUMBER': [u'1860', u'2008.'],
  'OTHER': [],
  'PERSON': [u'Scott']},
 {'LOCATION': [], 'NUMBER': [u'1857,'], 'OTHER': [], 'PERSON': []},
 {'LOCATION': [],
  'NUMBER': [u'1877,'],
  'OTHER': [],
  'PERSON': [u'Thomas Edison']},
 {'LOCATION': [], 'NUMBER': [], 'OTHER': [], 'PERSON': []},
 {'LOCATION': [], 'NUMBER': [], 'OTHER': [], 'PERSON': []},
 {'LOCATION': [], 'NUMBER': [], 'OTHER': [u'Edison'], 'PERSON': []},
 {'LOCATION': [], 'NUMBER': [], 'OTHER': [

# Answer Ranking

In [60]:
import gensim
from gensim import corpora
import math

class BM25 :
    def __init__(self, fn_docs, delimiter='|') :
        self.dictionary = corpora.Dictionary()
        self.DF = {}
        self.delimiter = delimiter
        self.DocTF = []
        self.DocIDF = {}
        self.N = 0
        self.DocAvgLen = 0
        self.fn_docs = fn_docs
        self.DocLen = []
        self.buildDictionary()
        self.TFIDF_Generator()

    def buildDictionary(self) :
        raw_data = []
        for line in file(self.fn_docs) :
            raw_data.append(line.strip().split(self.delimiter))
        self.dictionary.add_documents(raw_data)

    def TFIDF_Generator(self, base=math.e) :
        docTotalLen = 0
        for line in file(self.fn_docs) :
            doc = line.strip().split(self.delimiter)
            docTotalLen += len(doc)
            self.DocLen.append(len(doc))
            #print self.dictionary.doc2bow(doc)
            bow = dict([(term, freq*1.0/len(doc)) for term, freq in self.dictionary.doc2bow(doc)])
            for term, tf in bow.items() :
                if term not in self.DF :
                    self.DF[term] = 0
                self.DF[term] += 1
            self.DocTF.append(bow)
            self.N = self.N + 1
        for term in self.DF:
            self.DocIDF[term] = math.log((self.N - self.DF[term] +0.5) / (self.DF[term] + 0.5), base)
        self.DocAvgLen = docTotalLen / self.N

    def BM25Score(self, Query=[], k1=1.5, b=0.75) :
        query_bow = self.dictionary.doc2bow(Query)
        scores = []
        for idx, doc in enumerate(self.DocTF) :
            commonTerms = set(dict(query_bow).keys()) & set(doc.keys())
            tmp_score = []
            doc_terms_len = self.DocLen[idx]
            for term in commonTerms :
                upper = (doc[term] * (k1+1))
                below = ((doc[term]) + k1*(1 - b + b*doc_terms_len/self.DocAvgLen))
                tmp_score.append(self.DocIDF[term] * upper / below)
            scores.append(sum(tmp_score))
        return scores

    def TFIDF(self) :
        tfidf = []
        for doc in self.DocTF :
            doc_tfidf  = [(term, tf*self.DocIDF[term]) for term, tf in doc.items()]
            doc_tfidf.sort()
            tfidf.append(doc_tfidf)
        return tfidf

    def Items(self) :
        # Return a list [(term_idx, term_desc),]
        items = self.dictionary.items()
        items.sort()
        return items

if __name__ == '__main__' :
    fn_docs = 'mycorpus.txt'
    bm25 = BM25(fn_docs, delimiter=' ')
    Query = 'The intersection graph of paths in trees survey Graph'
    Query = Query.split()
    scores = bm25.BM25Score(Query)
    print scores
    tfidf = bm25.TFIDF()
#     print bm25.Items()
    for i, tfidfscore in enumerate(tfidf):
        print i, tfidfscore

[0, -0.00993319335279988, 0.1712756703100223, -0.10833186147108911, -0.08897894166003212, 0.13457374095787467, 1.642922484773619, 0.15791141726235663, 1.0831388761516576]
0 [(0, 0.2168251319235133), (1, 0.2168251319235133), (2, 0.2168251319235133), (3, 0.2168251319235133), (4, 0.2168251319235133), (5, 0.13732653608351372), (6, 0.2168251319235133), (7, 0.13732653608351372)]
1 [(5, 0.10986122886681099), (8, 0.10986122886681099), (9, -0.1238078416812447), (10, 0.06190392084062235), (11, 0.10986122886681099), (12, 0.06190392084062235), (13, 0.10986122886681099), (14, 0.17346010553881064), (15, 0.10986122886681099)]
2 [(7, 0.1831020481113516), (10, 0.10317320140103725), (12, 0.10317320140103725), (16, 0.10317320140103725), (17, 0.28910017589801773), (18, 0.1831020481113516)]
3 [(9, -0.07737990105077794), (10, 0.07737990105077794), (18, 0.13732653608351372), (19, 0.13732653608351372), (20, 0.2168251319235133), (21, 0.2168251319235133), (22, 0.2168251319235133), (23, 0.2168251319235133)]
4 [(